# Installation

In [ ]:
# pip install gensim requests rouge bert_score openpyxl prettytable nltk gdown

In [ ]:
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('punkt_tab')

In [2]:
# import gdown
# import gzip
# import shutil
# import os

# compressed_path = "./GoogleNews-vectors-negative300.bin.gz"
# decompressed_path = "./GoogleNews-vectors-negative300.bin"


# # Google Drive file ID extracted from the link
# file_id = "0B7XkCwpI5KDYNlNUTTlSS21pQmM"

# # Download the file
# gdown.download(f"https://drive.google.com/uc?id={file_id}&export=download", compressed_path, quiet=False)

# print("Download completed.")


# # Extract the .bin file from .gz
# with gzip.open(compressed_path, "rb") as f_in:
#     with open(decompressed_path, "wb") as f_out:
#         shutil.copyfileobj(f_in, f_out)

# print("Extraction completed.")

# # (Optional) Delete the .gz file after extraction
# os.remove(compressed_path)
# print("Deleted the compressed file.")

# Performance Evaluation

In [1]:
import pandas as pd
import warnings
import logging

warnings.filterwarnings("ignore")
logging.getLogger('transformers').setLevel(logging.ERROR)

# Set the logging level to ERROR to ignore warnings
logging.getLogger("transformers").setLevel(logging.ERROR)


In [2]:
DATASET = "FoCus"                                             

# LLM = "Qwen2-5B-Benchmark"     
# LLM = "Qwen2-5B-DPO-AVG"     
LLM = "Qwen2-5B-DPO-LENGTH-PRIOR"   
                              
COT_SETUP = False

In [3]:
df = pd.read_csv(f'./Prompts/{DATASET}.csv')
print("Shape:", df.shape)

df.head(2)

Shape: (1000, 3)


,personas,context,act_response
0,I would like to visit the Nazareth House again...,User1: I think Ive been there before but I don...,User2: The history of the house you are intere...
1,I have been to Vermont a few times to go skiin...,"User1: Wow, this is amazing! What is this?\nUs...",User2: This house was use as a stop for slaves...


In [4]:
### Only For: FoCus, IT-ConvAI2

df['act_response'] = df['act_response'].apply(lambda x: x.split(':', 1)[1].strip() if ':' in x else x.strip())

print(df.isnull().sum())
df.head(2)

personas        0
context         0
act_response    0
dtype: int64


,personas,context,act_response
0,I would like to visit the Nazareth House again...,User1: I think Ive been there before but I don...,The history of the house you are interested in...
1,I have been to Vermont a few times to go skiin...,"User1: Wow, this is amazing! What is this?\nUs...",This house was use as a stop for slaves trying...


In [5]:
COT_ = "-COT" if COT_SETUP else ""

response = pd.read_csv(f'Responses/{DATASET}/{LLM}{COT_}.csv')
print("Shape:", response.shape)

print("\nMissing Values:")
print(response.isnull().sum())

response

Shape: (1000, 2)

Missing Values:
gen_response     203
response_time      0
dtype: int64


,gen_response,response_time
0,The Nazareth House is a historic institution l...,0.514938
1,The Truman Galusha House is a historic house l...,0.462074
2,The Marion Palace Theatre is a movie palace lo...,0.422826
3,Technische Universität Darmstadt is a renowned...,0.634715
4,Here's a suggestion for visiting England: Bost...,0.444070
...,...,...
995,NaN,0.884238
996,"Sure, let me provide you with a personalized r...",0.409802
997,NaN,0.899150
998,"Nyanga National Park is located in Botswana, a...",0.517449


In [6]:
# Calculate maximum number of words in each column
max_response_length = response['gen_response'].dropna().apply(lambda x: len(x.split())).max()

print(f"Maximum Response Length (in words): {max_response_length}")

Maximum Response Length (in words): 71


In [7]:
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Initialize stop words
stop_words = set(stopwords.words('english'))

# Function to preprocess text
def preprocess_text(text, remove_stop_words=True):
    if pd.isnull(text):
        return None
    text = text.lower()  # Lowercasing
    text = text.translate(str.maketrans('', '', string.punctuation))  # Removing punctuation
    tokens = word_tokenize(text)  # Tokenization
    if remove_stop_words:
        tokens = [word for word in tokens if word not in stop_words]  # Removing stop words
    return ' '.join(tokens)  # Join tokens back into a single string


# Create eval_df
eval_df = pd.DataFrame({
    'personas': df['personas'],
    'context': df['context'],
    'act_response': df['act_response'],
    'gen_response': response['gen_response'],
    'response_time': response['response_time']
})

print(eval_df.isnull().sum())
eval_df.head()

personas           0
context            0
act_response       0
gen_response     203
response_time      0
dtype: int64


,personas,context,act_response,gen_response,response_time
0,I would like to visit the Nazareth House again...,User1: I think Ive been there before but I don...,The history of the house you are interested in...,The Nazareth House is a historic institution l...,0.514938
1,I have been to Vermont a few times to go skiin...,"User1: Wow, this is amazing! What is this?\nUs...",This house was use as a stop for slaves trying...,The Truman Galusha House is a historic house l...,0.462074
2,I am fascinated by the Spanish Colonial Reviva...,"User1: Wow, this is amazing! What is this?\nUs...","Sure, you will like to know that this place wa...",The Marion Palace Theatre is a movie palace lo...,0.422826
3,I want to become a college student.I want to s...,User1: Where is this place?\nUser2: Hello! Wel...,Technische Universität Darmstadt in the top 25...,Technische Universität Darmstadt is a renowned...,0.634715
4,I like to visit england.I love church.I would ...,User1: Where is this place?\nUser2: This place...,"I suggest a place, for your wish of see librar...",Here's a suggestion for visiting England: Bost...,0.444070


In [8]:
import torch
device = 0 if torch.cuda.is_available() else -1  # device set to 0 for GPU, -1 for CPU

In [9]:
import pandas as pd
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
import bert_score
from tqdm import tqdm
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gensim
from nltk.corpus import stopwords
from transformers import pipeline, BertForSequenceClassification, BertTokenizer

import sys
import os

# Add UniEval to PYTHONPATH
sys.path.append(os.path.abspath("UniEval"))  # Update with your actual path

from UniEval.utils import convert_to_json
from UniEval.metric.evaluator import get_evaluator



# Initialize ROUGE scorer
rouge = Rouge()

# Lists to store the metrics
ue_scores = []
c_scores = []
consistency_scores = []
persona_distance_scores = []
coh_unieval_scores = []


bert_snli_dir = "Fine-tuning/output/bert_snli"
bert_snli_model = BertForSequenceClassification.from_pretrained(bert_snli_dir)
bert_snli_tokenizer = BertTokenizer.from_pretrained(bert_snli_dir)

# Initialize the NLI pipeline for UE Score
bert_on_snli = pipeline('text-classification', model = bert_snli_model, tokenizer = bert_snli_tokenizer, device=0)

bert_dnli_dir = "Fine-tuning/output/bert_dnli"
bert_dnli_model = BertForSequenceClassification.from_pretrained(bert_dnli_dir)
bert_dnli_tokenizer = BertTokenizer.from_pretrained(bert_dnli_dir)

# Initialize the NLI pipeline
bert_on_dnli = pipeline('text-classification', model = bert_dnli_model, tokenizer = bert_dnli_tokenizer, device=0)


# Initialize the Word2Vec Model
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format("./GoogleNews-vectors-negative300.bin", binary=True)
stop_words = set(stopwords.words('english'))


# Initialize smoothing function
smoothing_function = SmoothingFunction().method1

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

def calculate_c_score(gen_response, persona):
    """
    Calculate the C score based on the entailment results between a generated response (R)
    and a given persona (P).

    Returns:
    int: C-score with possible values:
         1 for entailment (positive),
         0 for neutral,
         -1 for contradiction (negative).
    """

    # Define the label mapping to interpret the NLI model's output
    label_mapping = {
        'LABEL_0': 'negative',
        'LABEL_1': 'neutral',
        'LABEL_2': 'positive'
    }
    
    # Check entailment between persona (P) and generated response (R)
    result_pr = bert_on_dnli(f"{persona} {gen_response}")
    label_pr = label_mapping.get(result_pr[0]['label'], 'unknown')

    # Determine C score based on entailment results
    if label_pr == 'positive':
        return 1
    elif label_pr == 'neutral':
        return 0
    elif label_pr == 'negative':
        return -1
    else:
        raise ValueError(f"Unexpected label encountered: {label_pr}")


def calculate_consistency_score(gen_response, persona):
    """
    Calculate the Consistency Score based on the binary entailment results 
    between a generated response (R) and a given persona (P).

    Returns:
    int: Consistency Score with binary values:
         1 for entailment or neutral,
         0 for contradiction.
    """

    # Define the label mapping for binary classification
    label_mapping = {
        'LABEL_0': 'negative',
        'LABEL_1': 'neutral',
        'LABEL_2': 'positive'
    }

    # Check entailment between persona (P) and generated response (R)
    result_pr = bert_on_dnli(f"{persona} {gen_response}")
    label_pr = label_mapping.get(result_pr[0]['label'], 'unknown')

    # Determine Consistency Score based on binary entailment results
    if label_pr in ['positive', 'neutral']:
        return 1
    elif label_pr == 'negative':
        return 0
    else:
        raise ValueError(f"Unexpected label encountered: {label_pr}")


def calculate_ue_score(act_response, gen_response, persona):
    """
    Calculate the UE score based on entailment between persona, actual response, and generated response.

    Returns:
    int: UE score with possible values 2, 1, or 0.
    """

    # Define the label mapping to interpret the NLI model's output
    label_mapping = {
        'LABEL_0': 'entailment',
        'LABEL_1': 'neutral',
        'LABEL_2': 'contradiction'
    }
    
    # Check entailment between persona (P) and generated response (R)
    result_pr = bert_on_snli(f"{persona} [SEP] {gen_response}")
    label_pr = label_mapping.get(result_pr[0]['label'], 'unknown')

    # Check entailment between actual response (Q) and generated response (R)
    result_qr = bert_on_snli(f"{act_response} [SEP] {gen_response}")
    label_qr = label_mapping.get(result_qr[0]['label'], 'unknown')

    # Determine UE score based on entailment results
    if label_pr == 'entailment' and label_qr == 'entailment':
        return 2
    elif label_pr == 'entailment':
        return 1
    else:
        return 0


def calculate_coh_unieval_score(personas, contexts, gen_responses):
    """
    Args:
        personas (list): List of persona information as additional context.
        contexts (str or list): Conversation histories leading to the responses.
        gen_responses (str or list): Generated responses to be evaluated.

    Returns:
        float: The coherence score.
    """
    
    # Ensure personas is a list and flatten if necessary
    personas = [' '.join(p) if isinstance(p, list) else p for p in personas]

    # Ensure contexts and gen_responses are lists
    if isinstance(contexts, str):
        contexts = [contexts]  # Convert single string to list

    if isinstance(gen_responses, str):
        gen_responses = [gen_responses]  # Convert single string to list

    # Prepare inputs for UniEval
    data = convert_to_json(output_list=gen_responses, src_list=contexts, context_list=personas)

    # Initialize the evaluator for dialogue tasks
    evaluator = get_evaluator('dialogue')

    # Evaluate and obtain scores for all inputs
    eval_scores = evaluator.evaluate(data, print_result=False)
    
    # Extract and return only the first coherence score
    return eval_scores[0].get("coherence", None) if eval_scores else None



def compute_persona_distance(persona, response, model, stop_words):
    # Tokenize and filter stopwords
    persona_tokens = [word for word in persona.lower().split() if word not in stop_words]
    response_tokens = [word for word in response.lower().split() if word not in stop_words]
    
    # Get word vectors
    persona_vecs = [model[word] for word in persona_tokens if word in model]
    response_vecs = [model[word] for word in response_tokens if word in model]
    
    # If no vectors found, return zero similarity
    if not persona_vecs or not response_vecs:
        return 0.0
    
    # Compute average vectors
    persona_avg_vec = np.mean(persona_vecs, axis=0)
    response_avg_vec = np.mean(response_vecs, axis=0)
    
    # Compute cosine similarity
    return cosine_similarity([persona_avg_vec], [response_avg_vec])[0][0]



Device set to use cuda:0
Device set to use cuda:0


In [10]:
import logging

# Set the logging level to ERROR to suppress warnings about training
logging.getLogger("transformers").setLevel(logging.ERROR)

# Default worst-case values
worst_c_score = -1.0
worst_consistency_score = 0.0
worst_idf_score = 0.0
worst_ue_score = 0.0
worst_persona_distance_score = 0.0
worst_coh_unieval_score = 0.0

In [11]:
# Iterate over each row
for index, row in tqdm(eval_df.iterrows(), total=len(eval_df)):
    personas = row['personas']
    contexts = row['context']
    act_response = row['act_response']
    gen_response = row['gen_response']
    
    # Check for NaN or None in gen_response
    if pd.isna(gen_response):
    
        c_scores.append(worst_c_score)
        coh_unieval_scores.append(worst_coh_unieval_score)
        persona_distance_scores.append(worst_persona_distance_score)
        ue_scores.append(worst_ue_score)

        continue

    
    coh_unieval_scores.append(calculate_coh_unieval_score(personas, contexts, gen_response)) 
    
    c_scores.append(calculate_c_score(personas, gen_response))
    
    ue_scores.append(calculate_ue_score(act_response, gen_response, personas))                  

    persona_distance_scores.append(compute_persona_distance(personas, gen_response, word2vec_model,
                                                            stop_words))


# Compile metrics into DataFrame
metrics_df = pd.DataFrame({
    'Coh-UniEval': coh_unieval_scores,
    'C Score': c_scores,
    'UE Score': ue_scores,
    'Persona Distance': persona_distance_scores
})

metrics_df

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [38:41<00:00,  2.32s/it]


,Coh-UniEval,C Score,UE Score,Persona Distance
0,0.994554,0.0,0.0,0.330360
1,0.993552,0.0,0.0,0.548835
2,0.992067,0.0,0.0,0.477514
3,0.996803,0.0,0.0,0.553215
4,0.992959,0.0,2.0,0.548066
...,...,...,...,...
995,0.000000,-1.0,0.0,0.000000
996,0.068506,0.0,0.0,0.273092
997,0.000000,-1.0,0.0,0.000000
998,0.997147,0.0,1.0,0.398505


In [12]:
# # Storing the full results
output_path = f'./Metrics Results/{DATASET}/{LLM}{COT_}-results.xlsx'

df_concat = pd.concat([eval_df, metrics_df], axis=1)

df_concat.to_excel(output_path, index=False)

# Load and Aggregate Results

In [13]:
import pandas as pd

df = pd.read_excel(f"Metrics Results/{DATASET}/{LLM}-results.xlsx")
metrics_df = df.drop(columns=["personas", "context", "act_response","gen_response"])
metrics_df

,response_time,Coh-UniEval,C Score,UE Score,Persona Distance
0,0.514938,0.994554,0,0,0.330360
1,0.462074,0.993552,0,0,0.548835
2,0.422826,0.992067,0,0,0.477514
3,0.634715,0.996803,0,0,0.553215
4,0.444070,0.992959,0,2,0.548066
...,...,...,...,...,...
995,0.884238,0.000000,-1,0,0.000000
996,0.409802,0.068506,0,0,0.273092
997,0.899150,0.000000,-1,0,0.000000
998,0.517449,0.997147,0,1,0.398505


In [14]:
# Calculate the mean (average) and standard deviation, rounded to 2 decimal places
avg_values = metrics_df.mean().round(2)
std_values = metrics_df.std(ddof=0).round(2)  # Use ddof=0 for population standard deviation

# Combine the average and standard deviation into the format "avg ± std"
combined_values = avg_values.astype(str) + " ± " + std_values.astype(str)

# Insert the LLM name at the beginning of the combined values
combined_values = combined_values.tolist()
combined_values.insert(0, LLM)

# Create a DataFrame for the combined average ± std row
result_df = pd.DataFrame([combined_values], columns=['Model'] + metrics_df.columns.tolist())

# Add the ratio of invalid gen_response
invalid_gen_res_ratio = df['gen_response'].isna().sum() /len(df) 

result_df['Failure Ratio'] = f"{round(invalid_gen_res_ratio, 3)} ± 0.00"  # No std for Failure Ratio

result_df

,Model,response_time,Coh-UniEval,C Score,UE Score,Persona Distance,Failure Ratio
0,Qwen2-5B-DPO-LENGTH-PRIOR,0.64 ± 0.18,0.73 ± 0.44,-0.07 ± 0.7,0.3 ± 0.64,0.37 ± 0.23,0.203 ± 0.00


In [15]:
# Load the existing Excel file and update or append the average row
output_path = f'./Evaluations/{DATASET}{COT_}-results.xlsx'

try:
    # Load existing data
    existing_df = pd.read_excel(output_path)
    # Check if the model name already exists
    if LLM in existing_df['Model'].values:
        # Update the row with the same model name
        existing_df.loc[existing_df['Model'] == LLM, :] = result_df.values
    else:
        # Append the new data
        existing_df = pd.concat([existing_df, result_df], ignore_index=True)
except FileNotFoundError:
    # If the file does not exist, create a new DataFrame
    existing_df = result_df

# Save the updated DataFrame to an Excel file
existing_df.to_excel(output_path, index=False)

existing_df

,Model,response_time,Coh-UniEval,C Score,UE Score,Persona Distance,Failure Ratio
0,Qwen2-5B-Benchmark,0.77 ± 0.16,0.6 ± 0.49,-0.24 ± 0.76,0.14 ± 0.45,0.3 ± 0.26,0.39 ± 0.00
1,Qwen2-5B-DPO-AVG,0.83 ± 0.09,0.39 ± 0.48,-0.47 ± 0.76,0.16 ± 0.48,0.22 ± 0.28,0.595 ± 0.00
2,Qwen2-5B-DPO-LENGTH-PRIOR,0.64 ± 0.18,0.73 ± 0.44,-0.07 ± 0.7,0.3 ± 0.64,0.37 ± 0.23,0.203 ± 0.00


## Reviwing the Results

In [16]:
DATASET = "FoCus"  
# COT_ = "-COT"
COT_ =  ""

response = pd.read_excel(f'./Evaluations/{DATASET}{COT_}-results.xlsx')
response

,Model,response_time,Coh-UniEval,C Score,UE Score,Persona Distance,Failure Ratio
0,Qwen2-5B-Benchmark,0.77 ± 0.16,0.6 ± 0.49,-0.24 ± 0.76,0.14 ± 0.45,0.3 ± 0.26,0.39 ± 0.00
1,Qwen2-5B-DPO-AVG,0.83 ± 0.09,0.39 ± 0.48,-0.47 ± 0.76,0.16 ± 0.48,0.22 ± 0.28,0.595 ± 0.00
2,Qwen2-5B-DPO-LENGTH-PRIOR,0.64 ± 0.18,0.73 ± 0.44,-0.07 ± 0.7,0.3 ± 0.64,0.37 ± 0.23,0.203 ± 0.00
